In [98]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from mpi4py import MPI
import numpy as np
import pandas as pd
import operator
np.random.seed(100)

In [99]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
p = comm.Get_size()

In [100]:
# Definicion de dataset
dataset = datasets.load_iris()
X, y = dataset.data[:, 1:3], dataset.target
# Clasificadores que vamos a comparar
c1 = LogisticRegression()
c2 = GaussianNB()

In [101]:
class VotingClassifierMPI(BaseEstimator, ClassifierMixin):
    
    def __init__(self, classifiers, weights=None):
        self.classifiers = classifiers
        self.weights = weights
    
    def predict_proba(self, X):
        self.probas_ = [classifiers.predict_proba(X) for classifiers in self.classifiers]
        return np.average(self.probas_, axis=0, weights=self.weights)
        
    def fit(self, X, y):
        # intento de morir
        if rank == 0:
            for classifiers in self.classifiers:
                classifiers.fit(X, y)
                comm.send(classifiers, dest=0, tag=11)
        else:        
            for proc in range(1, p):
                comm.recv(source=proc, tag=11)
                
    def predict(self, X):
        # todavia nada
        self.classes_ = np.asarray([classifiers.predict(X) for classifiers in self.classifiers])
        if self.weights:
            average = self.predict_proba(X)
            m_voting = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=average)
        else:
            m_voting = np.asarray([np.argmax(np.bincount(self.classes[:,i])) for i in range(self.classes_.shape[1])])
        return m_voting

In [102]:
class VotingClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, classifiers, weights=None):
        self.classifiers = classifiers
        self.weights = weights
    
    def predict_proba(self, X):
        self.probas_ = [classifiers.predict_proba(X) for classifiers in self.classifiers]
        return np.average(self.probas_, axis=0, weights=self.weights)
        
    def fit(self, X, y):
        for classifiers in self.classifiers:
            classifiers.fit(X, y)
    
    def predict(self, X):
        self.classes_ = np.asarray([classifiers.predict(X) for classifiers in self.classifiers])
        if self.weights:
            average = self.predict_proba(X)
            m_voting = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=average)
        else:
            m_voting = np.asarray([np.argmax(np.bincount(self.classes[:,i])) for i in range(self.classes_.shape[1])])
        return m_voting

In [103]:
voting_clf = VotingClassifier(classifiers=[c1, c2])
voting_clf_w = VotingClassifier(classifiers=[c1, c2], weights=[1, 1])
voting_clf_mpi = VotingClassifierMPI(classifiers=[c1, c2], weights=[1, 1])

In [104]:
i = 0
df = pd.DataFrame(columns=('w1', 'w2', 'mean', 'std'))
for w1 in range(1, 4):
    for w2 in range(1, 4):
        if (len(set((w1, w2))) == 1):
            continue
        #t_start = MPI.Wtime()
        scores = cross_val_score(estimator=voting_clf_w, X=X, y=y, cv=5, scoring='accuracy', n_jobs=1)
        #t_diff = MPI.Wtime()-t_start 
        df.loc[i] = [w1, w2, scores.mean(), scores.std()]
        i += 1
df.sort_values(by=['mean', 'std'], ascending=False)
#print(t_diff)

/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/giogiasvalley/opt/a

,w1,w2,mean,std
0,1.0,2.0,0.926667,0.044222
1,1.0,3.0,0.926667,0.044222
2,2.0,1.0,0.926667,0.044222
3,2.0,3.0,0.926667,0.044222
4,3.0,1.0,0.926667,0.044222
5,3.0,2.0,0.926667,0.044222


In [105]:
i = 0
df = pd.DataFrame(columns=('w1', 'w2', 'mean', 'std'))
for w1 in range(1, 4):
    for w2 in range(1, 4):
        if (len(set((w1, w2))) == 1):
            continue
        #t_start = MPI.Wtime()
        scores = cross_val_score(estimator=voting_clf_mpi, X=X, y=y, cv=5, scoring='accuracy', n_jobs=1)
        #t_diff = MPI.Wtime()-t_start 
        df.loc[i] = [w1, w2, scores.mean(), scores.std()]
        i += 1
df.sort_values(by=['mean', 'std'], ascending=False)
#print(t_diff)

/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/giogiasvalley/opt/a

/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/giogiasvalley/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


,w1,w2,mean,std
0,1.0,2.0,0.926667,0.044222
1,1.0,3.0,0.926667,0.044222
2,2.0,1.0,0.926667,0.044222
3,2.0,3.0,0.926667,0.044222
4,3.0,1.0,0.926667,0.044222
5,3.0,2.0,0.926667,0.044222
